# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 28 2022 3:32PM,240672,19,ADV80003630,"AdvaGen Pharma, Ltd",Released
1,Jun 28 2022 3:32PM,240672,19,ADV80003631,"AdvaGen Pharma, Ltd",Released
2,Jun 28 2022 3:32PM,240672,19,ADV80003632,"AdvaGen Pharma, Ltd",Released
3,Jun 28 2022 3:32PM,240672,19,ADV80003633,"AdvaGen Pharma, Ltd",Released
4,Jun 28 2022 3:32PM,240672,19,ADV80003634,"AdvaGen Pharma, Ltd",Released
5,Jun 28 2022 3:32PM,240672,19,ADV80003635,"AdvaGen Pharma, Ltd",Released
6,Jun 28 2022 3:32PM,240672,19,ADV80003636,"AdvaGen Pharma, Ltd",Released
7,Jun 28 2022 3:32PM,240672,19,ADV80003637,"AdvaGen Pharma, Ltd",Released
8,Jun 28 2022 3:32PM,240672,19,ADV80003638,"AdvaGen Pharma, Ltd",Released
9,Jun 28 2022 3:32PM,240672,19,ADV80003639,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
41,240668,Released,3
42,240669,Released,19
43,240671,Released,1
44,240672,Released,32
45,240673,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240668,NaN,NaN,3.0
240669,NaN,NaN,19.0
240671,NaN,NaN,1.0
240672,NaN,NaN,32.0
240673,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240574,0.0,0.0,1.0
240577,0.0,0.0,1.0
240579,0.0,0.0,1.0
240589,0.0,15.0,11.0
240591,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240574,0,0,1
240577,0,0,1
240579,0,0,1
240589,0,15,11
240591,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240574,0,0,1
1,240577,0,0,1
2,240579,0,0,1
3,240589,0,15,11
4,240591,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240574,,,1
1,240577,,,1
2,240579,,,1
3,240589,,15,11
4,240591,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 28 2022 3:32PM,240672,19,"AdvaGen Pharma, Ltd"
32,Jun 28 2022 3:31PM,240673,20,Sartorius Stedim North America
33,Jun 28 2022 3:28PM,240671,10,"Citieffe, Inc."
34,Jun 28 2022 3:27PM,240669,10,ISDIN Corporation
53,Jun 28 2022 3:22PM,240668,10,"Methapharm, Inc."
56,Jun 28 2022 3:22PM,240667,10,"Methapharm, Inc."
64,Jun 28 2022 3:11PM,240665,22,"NBTY Global, Inc."
65,Jun 28 2022 3:08PM,240664,20,"Exact-Rx, Inc."
66,Jun 28 2022 3:02PM,240663,22,"NBTY Global, Inc."
67,Jun 28 2022 2:58PM,240662,18,"ClearSpec, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 28 2022 3:32PM,240672,19,"AdvaGen Pharma, Ltd",,,32
1,Jun 28 2022 3:31PM,240673,20,Sartorius Stedim North America,,,1
2,Jun 28 2022 3:28PM,240671,10,"Citieffe, Inc.",,,1
3,Jun 28 2022 3:27PM,240669,10,ISDIN Corporation,,,19
4,Jun 28 2022 3:22PM,240668,10,"Methapharm, Inc.",,,3
5,Jun 28 2022 3:22PM,240667,10,"Methapharm, Inc.",,,8
6,Jun 28 2022 3:11PM,240665,22,"NBTY Global, Inc.",,,1
7,Jun 28 2022 3:08PM,240664,20,"Exact-Rx, Inc.",,,1
8,Jun 28 2022 3:02PM,240663,22,"NBTY Global, Inc.",,,1
9,Jun 28 2022 2:58PM,240662,18,"ClearSpec, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 28 2022 3:32PM,240672,19,"AdvaGen Pharma, Ltd",32,,
1,Jun 28 2022 3:31PM,240673,20,Sartorius Stedim North America,1,,
2,Jun 28 2022 3:28PM,240671,10,"Citieffe, Inc.",1,,
3,Jun 28 2022 3:27PM,240669,10,ISDIN Corporation,19,,
4,Jun 28 2022 3:22PM,240668,10,"Methapharm, Inc.",3,,
5,Jun 28 2022 3:22PM,240667,10,"Methapharm, Inc.",8,,
6,Jun 28 2022 3:11PM,240665,22,"NBTY Global, Inc.",1,,
7,Jun 28 2022 3:08PM,240664,20,"Exact-Rx, Inc.",1,,
8,Jun 28 2022 3:02PM,240663,22,"NBTY Global, Inc.",1,,
9,Jun 28 2022 2:58PM,240662,18,"ClearSpec, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 28 2022 3:32PM,240672,19,"AdvaGen Pharma, Ltd",32,,
1,Jun 28 2022 3:31PM,240673,20,Sartorius Stedim North America,1,,
2,Jun 28 2022 3:28PM,240671,10,"Citieffe, Inc.",1,,
3,Jun 28 2022 3:27PM,240669,10,ISDIN Corporation,19,,
4,Jun 28 2022 3:22PM,240668,10,"Methapharm, Inc.",3,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 28 2022 3:32PM,240672,19,"AdvaGen Pharma, Ltd",32.0,NaN,NaN
1,Jun 28 2022 3:31PM,240673,20,Sartorius Stedim North America,1.0,NaN,NaN
2,Jun 28 2022 3:28PM,240671,10,"Citieffe, Inc.",1.0,NaN,NaN
3,Jun 28 2022 3:27PM,240669,10,ISDIN Corporation,19.0,NaN,NaN
4,Jun 28 2022 3:22PM,240668,10,"Methapharm, Inc.",3.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 28 2022 3:32PM,240672,19,"AdvaGen Pharma, Ltd",32.0,0.0,0.0
1,Jun 28 2022 3:31PM,240673,20,Sartorius Stedim North America,1.0,0.0,0.0
2,Jun 28 2022 3:28PM,240671,10,"Citieffe, Inc.",1.0,0.0,0.0
3,Jun 28 2022 3:27PM,240669,10,ISDIN Corporation,19.0,0.0,0.0
4,Jun 28 2022 3:22PM,240668,10,"Methapharm, Inc.",3.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1684534,39.0,19.0,7.0
15,1203063,16.0,44.0,10.0
18,721934,3.0,0.0,0.0
19,962465,45.0,15.0,0.0
20,1684487,47.0,3.0,4.0
21,1203047,4.0,1.0,0.0
22,1443844,6.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1684534,39.0,19.0,7.0
1,15,1203063,16.0,44.0,10.0
2,18,721934,3.0,0.0,0.0
3,19,962465,45.0,15.0,0.0
4,20,1684487,47.0,3.0,4.0
5,21,1203047,4.0,1.0,0.0
6,22,1443844,6.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,39.0,19.0,7.0
1,15,16.0,44.0,10.0
2,18,3.0,0.0,0.0
3,19,45.0,15.0,0.0
4,20,47.0,3.0,4.0
5,21,4.0,1.0,0.0
6,22,6.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,39.0
1,15,Released,16.0
2,18,Released,3.0
3,19,Released,45.0
4,20,Released,47.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,18,19,20,21,22
Status,,,,,,,
Completed,7.0,10.0,0.0,0.0,4.0,0.0,0.0
Executing,19.0,44.0,0.0,15.0,3.0,1.0,0.0
Released,39.0,16.0,3.0,45.0,47.0,4.0,6.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,18,19,20,21,22
0,Completed,7.0,10.0,0.0,0.0,4.0,0.0,0.0
1,Executing,19.0,44.0,0.0,15.0,3.0,1.0,0.0
2,Released,39.0,16.0,3.0,45.0,47.0,4.0,6.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,18,19,20,21,22
0,Completed,7.0,10.0,0.0,0.0,4.0,0.0,0.0
1,Executing,19.0,44.0,0.0,15.0,3.0,1.0,0.0
2,Released,39.0,16.0,3.0,45.0,47.0,4.0,6.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()